In [2]:
# google drive settings 
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# google drive settings 
%%capture
%cd /content/gdrive/My\ Drive/NLP

In [4]:
from collections import defaultdict
import numpy as np
import pandas as pd
import nltk

In [5]:
first_week = pd.read_csv("tweet_data/first_week.csv")
second_week = pd.read_csv("tweet_data/second_week.csv")
third_week = pd.read_csv("tweet_data/third_week.csv")
fourth_week = pd.read_csv("tweet_data/fourth_week.csv")

In [6]:
first_week = first_week[first_week.clean_text.isna()==False] # drop all tweets without text
second_week = second_week[second_week.clean_text.isna()==False] # drop all tweets without text
third_week = third_week[third_week.clean_text.isna()==False] # drop all tweets without text
fourth_week = fourth_week[fourth_week.clean_text.isna()==False] # drop all tweets without text

In [7]:
class LM:
 
  def __init__(self, smoothing, n_grams_size):
    self.smoothing = smoothing
    self.n_grams_size = n_grams_size
    self.counts = defaultdict(lambda: defaultdict(lambda: smoothing))
    self.START = '_***_'
    self.STOP = '_STOP_'
 
  def fit(self, document):
    corpus = [line.strip().split() for line in document]
  
    for sentence in corpus:
      tokens = [self.START for _ in range(self.n_grams_size)] + sentence + [self.STOP]
      for n_grams_tuple in nltk.ngrams(tokens, self.n_grams_size):
        self.counts[n_grams_tuple[:-1]][n_grams_tuple[-1]] += 1
  
  def _sample_next_word(self, history_tuple, iters=50):
    for _ in range(iters):
      keys, values = zip(*self.counts[history_tuple].items())
      values = np.array(values)
      values /= values.sum()
      sample = np.random.multinomial(1, values) 
    
    return keys[np.argmax(sample)]
  
  def generate(self, words_list = []):
    result = [self.START for _ in range(self.n_grams_size)]
    if words_list: 
      result.extend(words_list)

    history_tuple = tuple([result[i] for i in range(-self.n_grams_size+1, 0, 1)])
    next_word = self._sample_next_word(history_tuple)
    result.append(next_word)
  
    while next_word != self.STOP:
      history_tuple = tuple(result[i] for i in range(-self.n_grams_size+1, 0, 1))
      next_word = self._sample_next_word(history_tuple)
      result.append(next_word)
  
    return ' '.join(result[self.n_grams_size:-1])

In [8]:
def create_lm(dataset, n=4): 
  tweets = dataset['clean_tweet'].to_list()
  lm = LM(smoothing=0.001, n_grams_size=n)
  lm.fit(document=tweets)
  return lm 

## 7-gram Model

In [8]:
#create language model for each week 
w1 = create_lm(first_week, n=7)
w2 = create_lm(second_week, n=7)
w3 = create_lm(third_week, n=7)
w4 = create_lm(fourth_week, n=7)

In [9]:
number_of_tweets = 5

In [10]:
print(np.unique([w1.generate() for _ in range(number_of_tweets)]))

['_***_ _***_ coronavirus quarantena per chi in zone rosse in ultimi giorni'
 'a annullato convegno sul coronavirus'
 'coronavirus i consigli dei medici sportivi per chi fa sport all interno di palestre e palasport corriere dell umbria'
 'da vedere che insegna la struccante viso lisoformlavanda struccante occhi quasarlavavetri per andare a ballare se ti senti un po cagna due gocce di frontline dietro al collo adoro a lui e a gilda'
 'gestione delle emozioni e corretta gli antidoti del finanziario per arginare il panico da nel cliente investitore wallstreetita']


In [11]:
print(np.unique([w2.generate() for _ in range(number_of_tweets)]))

['_***_ _***_ _***_ _***_ _***_ giorno di quarantena mi sento strano sta cosa che non so cosa succederà mi blocca e destabilizza non so come organizzarmi e inquadrare la mia vita ho sempre visto la mia vita come un pezzetto del lego adesso sembra che il mio lego non abbia più forma mi sento perso'
 '_***_ _***_ _***_ coronavirus in lombardia rallenta il trend i decessi i contagi gellera più chiari che scuri a milano hotel requisito per la quarantena via repubblica'
 'coronavirus mattino cinque senza federica panicucci repliche forum e avanti un altro'
 'le uniche gioie di poter lavorare ai tempi del covid ad un metro di distanza a bit of joy in working in the era of covid one meter away'
 'sisal enternainment s p a emergenza coronavirus che il jackpot del superenalotto vada agli ospedali italiani firma la petizione via changeitalia']


In [12]:
print(np.unique([w3.generate() for _ in range(number_of_tweets)]))

['_***_ istituto superiore di sanità rischi dell acquisto on line di farmaci anti covid e danni delle fake news'
 '_***_ la mascherina di zaia ha pure il paraorecchie bella davvero coronavirus zaia mascherina sempre anche in strada campania via libera alle pizze a domicilio politica ansa'
 '_***_ sassee l angelo che non lascia soli i gatti randagi di new york ai tempi del coronavirus di lastampa'
 'siete in sette e io ho scelto lei pieralfredo rizzi da codogno a pavia scegliere chi far sopravvivere'
 'trovato un metodo per identificare la traccia genetica del nuovo coronavirus rivelandone gli spostamenti e l evoluzione']


In [13]:
print(np.unique([w4.generate() for _ in range(number_of_tweets)]))

['_***_ _***_ _***_ emergenza da covid e norme in materia di enti non profit'
 '_***_ _***_ coronavirus e fase nelle campagne solo lo dei contagi'
 '_***_ _***_ oristano reagiamo al covid con l e commerce'
 'la redazione il patron della virtusbo massimo zanetti è bloccato in polinesia da tre mesi sulla sua barca il noto imprenditore non può attualmente rientrare in italia a causa delle normative sul covid lo riferisce la tribuna'
 'rezza vaccino cerotto anti covid sarebbe utile ma aspettiamo i dati covid']


In [14]:
print(np.unique([w1.generate(['coronavirus', 'è']) for _ in range(number_of_tweets)]))

['coronavirus è assalto ai supermercati code alle casse e scaffali di acqua e gel vuoti l emergenza coronavirus vista dai supermercati di repubblica'
 'coronavirus è il caso di dirlo viva le élite'
 'coronavirus è il triste spettacolo offerto dall italia autolesionista'
 'coronavirus è massima allerta ospedali da campo e più tamponi'
 'coronavirus è ufficialmente psicosi controlli a due testimoni di lodi al tribunale di messina']


In [15]:
print(np.unique([w2.generate(['coronavirus', 'è']) for _ in range(number_of_tweets)]))

['coronavirus è assalto alla spesa online tutti i servizi bloccati'
 'coronavirus è grave e assurdo che siano state sottratte alla dogana in polonia oltre mila mascherine ffp acquistate dalla regionelazio abbiamo subito messo al corrente la protezione civile nazionale con l intervento successivo delle nostre autorità diplomatiche'
 'coronavirus è in corso l assemblea della lega di serie a'
 'coronavirus è morto vittorio gregotti'
 'coronavirus è stato il pipistrello via youtube']


In [16]:
print(np.unique([w3.generate(['coronavirus', 'è']) for _ in range(number_of_tweets)]))

['coronavirus è caos sardegna sulla cig il mise dalla regione zero domande'
 'coronavirus è guarito l ex sindaco di sabaudia salvatore schintu'
 'coronavirus è il giorno della cabina di regia tra governo ed enti locali su fase regioni comuni e province indicazioni chiare su trasporti e uffici decaro anci bisogna definire capienza su metro e bus'
 'coronavirus è siciliana la mascherina utilizzabile all infinito di negozio per le imprese news'
 'coronavirus è stato creato dall uomo ecco la prova un premio nobel fa discutere di moneypuntoit']


In [17]:
print(np.unique([w4.generate(['coronavirus', 'è']) for _ in range(number_of_tweets)]))

['coronavirus è ancora mistero sulle origini'
 'coronavirus è finita chiuso il reparto covid ecco la provincia italiana senza più contagi da giorni'
 'coronavirus è guerra tra virologi tarro querela burioni e due giornalisti desolanti effetti collaterali del covid'
 'coronavirus è guerra tra virologi tarro querela burioni e due giornalisti opera di denigrazione ai miei danni nuovo capitolo della querelle tarro burioni il virologo napoletano giulio tarro ha infatti incaricato il suo legale l avvocato carlo taormina'
 'coronavirus è guerra tra virologi tarro querela burioni e due giornalisti opera di denigrazione continua']


## 4-gram Model

In [9]:
#create language model for each week 
w1_n4 = create_lm(first_week, n=4)
w2_n4 = create_lm(second_week, n=4)
w3_n4 = create_lm(third_week, n=4)
w4_n4 = create_lm(fourth_week, n=4)

In [10]:
number_of_tweets = 5

In [11]:
print(np.unique([w1_n4.generate(['conte']) for _ in range(number_of_tweets)]))

['conte con i tamponi per il abbiamo esagerato scommettiamo che cominceranno a farli i laboratori privati pazzesco'
 'conte con i tamponi per il virus'
 'conte giuseppi è in difficoltà nella del perché le gattare non riescono a gestire un po meglio la diffusione di nuovi modelli organizzativi e culturali al passo con la è rinviato a data da destinarsi per evitare che il virus uscisse dalla cina ora non saremmo il terzo paese per contagi no ma buongiorno anche a te sandra ricordi la madre me lo disse e me lo riempite'
 'conte impugna l ordinanza il governo potrebbe limitare la libera circolazione al di infettare altre zone inneggiando al potere del denaro e del guadagno anche in situazioni così gravi'
 'conte è pronto a impatto forte del metro news']


In [12]:
print(np.unique([w2_n4.generate(['conte']) for _ in range(number_of_tweets)]))

['conte agli italiani sul coronavirus'
 'conte gualtieri lagarde van der leyden li abbiamo votati'
 'conte l emergenza coronavirus aderendo alla campagna non si ferma scopri il museoarcheoca qui continua a seguirci sul canale youtube ogni giorno nuovi video dai iscriviti guarda condividi'
 'conte ora faccia l unica cosa che ci è sfuggito qualcosa che abbiamo tralasciato o sbagliato non è possibile uscire e entrare'
 'conte sta passando per il salvatore della patria non basterà questa non arriva al panettone']


In [13]:
print(np.unique([w3_n4.generate(['conte']) for _ in range(number_of_tweets)]))

['conte frena sul mes e spinge per gli eurobond attacco alla germania'
 'conte frena sul mes e sui coronabond sono solo chiacchiere il mes è una cagata pazzesca semicit grazie a albertobagnai per informazione e umorismo'
 'conte ha detto che la è riuscita a controllare le persone si ribelleranno la profezia di cinque anni fa un virus sconosciuto le mascherine e ti mandano watsapp per sapere se siamo stati fortunati ma anche bravi e attenti nel nostro comune vista la mole di provvedimenti nazionali che dovranno fare lo stesso'
 'conte i nuovi provvedimenti entreranno in vigore le norme per limitare l epidemia di sta determinando un evoluzione del settore della ristorazione mette a punto un questionario online per capire i rischi del contact digitale fatto male via youtube'
 'conte il mes ha brutta fama non dimentichiamo la e rilancia i coronabond']


In [14]:
print(np.unique([w4_n4.generate(['conte']) for _ in range(number_of_tweets)]))

['conte e il consenso popolare il premier sopravvivrà al coronavirus di ilpost'
 'conte già provato decreto rilancio la precisazione lo stato d emergenza per covid non è stato fatto un errore che pesa enormemente sul bilancio dell epidemia e le crescenti disparità di classe coronavirus school closures impact billion children and remote learning is increasing inequality via conversationuk'
 'conte ha firmato il dpcm sulle aperture dal maggio spettacoli all aperto fino a persone all aperto'
 'conte lo avevamo capito da un pezzo si prenderà una polmonite che sarà diagnosticata come covid'
 'conte sul vaccino anti covid ecco i numeri del in fonte ministero della salute']


In [15]:
print(np.unique([w4_n4.generate(['conte']) for _ in range(5)]))

['conte con i filtri di instagram mentre parlava in conferenza stampa'
 'conte dice ancora no a per il prezioso supporto all'
 'conte e il governo cadde l inghilterra sopravvisse e vinse la guerra'
 'conte firma il dpcm diretta'
 'conte non è ancora tempo di movida party e assembramenti di qualunque genere linea dura del ministro dell interno lamorgese a pavia']
